In [23]:
# !pip install --upgrade openai --quiet

In [1]:
from openai import OpenAI
import os
import json
from pandas import DataFrame


In [2]:
client = OpenAI(api_key=os.getenv("TRAVEL_PLANNER_API_KEY"))

### Travel Planner Defined Tools

In [3]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [4]:
from tools.accommodations.apis import Accommodations
from tools.attractions.apis import Attractions
from tools.cities.apis import Cities
from tools.flights.apis import Flights
from tools.restaurants.apis import Restaurants
from tools.googleDistanceMatrix.apis import GoogleDistanceMatrix
from tools.cities.apis import Cities

/Users/jiahaoliu/miniconda3/envs/travelplanner/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
flight_api = Flights()

Flights API loaded.


In [6]:
accommodation_api = Accommodations()
attraction_api = Attractions()
restaurant_api = Restaurants()
google_dist_api = GoogleDistanceMatrix()
city_api = Cities()

Accommodations loaded.
Attractions loaded.
Restaurants loaded.
GoogleDistanceMatrix loaded.
Cities loaded.


### Upload Files to OpenAI

In [11]:
DATA_ROOT = "../database/" # download from google drive

file_paths = [os.path.join(DATA_ROOT, f) for f in ("accommodations/clean_accommodations_2022.csv",
                                        "attractions/attractions.csv",
                                        "flights/clean_Flights_2022.csv",
                                        "googleDistanceMatrix/distance.csv",
                                        "restaurants/clean_restaurant_2022.csv")]
file_paths[0]

'../database/accommodations/clean_accommodations_2022.csv'

In [14]:
local_file2openai = {}
for fp in file_paths:
  openai_file = client.files.create(
    file=open(fp, "rb"),
    purpose='assistants'
  )
  local_file2openai[fp] = openai_file.model_dump_json()

In [19]:
local_file2openai_json = {}
for k, v in local_file2openai.items():
    local_file2openai_json[k] = json.loads(v)

In [22]:
with open("./local_file2openai_json.json", "w") as f:
    json.dump(local_file2openai_json, f, indent=4)

### Create Assistant

In [40]:
with open("./local_file2openai_json.json", "r") as f:
    local_file2openai_json = json.load(f)
    
oai_file_ids = []
for k, v in local_file2openai_json.items():
    oai_file_ids.append(v['id'])
    
oai_file_ids

['file-7ZdgNA4HvwtCWwg16AAPUmEp',
 'file-siMa0C67mQM4qUn9AkBFjKKy',
 'file-z3S4sL0zvULQKGECUPKaPvu5',
 'file-GdM62ZJ8sVOOSXQznpkOjC2K',
 'file-4BBloZC5bfAiBvaWY4DQdMVu']

In [41]:
assistant = client.beta.assistants.create(
  name="mobi-fork-travelplanner",
  description="You are a helpful travel planner",
  model="gpt-4o",
  tools=[
    { "type": "code_interpreter" },
    {
      "type": "function",
      "function": {
        "name": "FlightSearch",
        "description": "A flight information retrieval tool. For exmaple FlightSearch('New York', 'London', '2022-10-01') would fetch flights from New York to London on October 1 , 2022.",
        "parameters": {
          "type": "object",
          "properties": {
            "origin": {
              "type": "string",
              "description": "The city you 'll be flying out from."
            },
            "destination": {
              "type": "string",
              "description": "The city you aim to reach ."
            },
            "departure_date": {
              "type": "string",
              "description": "The date of your travel in YYYY-MM-DD format."
            }
          },
          "required": ["origin", "destination", "departure_date"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "DistanceMatrix",
        "description": "Estimate the distance, time and cost between two cities. For example, Example : DistanceMatrix('Paris', 'Lyon', 'self-driving') would provide driving distance, time and cost between Paris and Lyon .",
        "parameters": {
          "type": "object",
          "properties": {
            "origin": {
              "type": "string",
              "description": "The departure city of your journey"
            },
            "destination": {
              "type": "string",
              "description": " The destination city of your journey"
            },
            "mode": {
              "type": "string",
              "enum": ["Self-driving", "Taxi"],
              "description": "The method of transportation."
            }
          },
          "required": ["origin", "destination", "mode"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "AccommodationSearch",
        "description": "Discover accommodations in your desired city. For example, AccommodationSearch('Rome') would present a list of hotel rooms in Rome.",
        "parameters": {
          "type": "object",
          "properties": {
            "city": {
              "type": "string",
              "description": "The name of the city where you're seeking accommodation."
            }
          },
          "required": ["city"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "RestaurantSearch",
        "description": "Explore dining options in a city of your choice. For example, RestaurantSearch('Tokyo') would show a curated list of restaurants in Tokyo.",
        "parameters": {
          "type": "object",
          "properties": {
            "city": {
              "type": "string",
              "description": "The name of the city where you're seeking restaurant."
            }
          },
          "required": ["city"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "AttractionSearch",
        "description": "Find attractions in a city of your choice. For example, AttractionSearch('London') would return attractions in London.",
        "parameters": {
          "type": "object",
          "properties": {
            "city": {
              "type": "string",
              "description": "The name of the city where you're seeking attractions."
            }
          },
          "required": ["city"]
        }
      }
    },
    {
      "type": "function",
      "function": {
        "name": "CitySearch",
        "description": "Find cities in a state of your choice. For example, CitySearch('California') would return cities in California.",
        "parameters": {
          "type": "object",
          "properties": {
            "state": {
              "type": "string",
              "description": "The name of the state where you're seeking cities."
            }
          },
          "required": ["state"]
        }
      }
    },
    # {
    #   "type": "function",
    #   "function": {
    #     "name": "NotebookWrite",
    #     "description": "Writes a new data entry into the Notebook tool with a short description. This tool should be used immediately after FlightSearch, AccommodationSearch, AttractionSearch, RestaurantSearch, or DistanceMatrix. Only the data stored in Notebook can be seen by Planner. So you should write all the information you need into Notebook. For example, NotebookWrite('Flights from Rome to Paris in 2022-02-01') would store the information of flights from Rome to Paris in 2022-02-01 in the Notebook.",
    #     "parameters": {
    #       "type": "object",
    #       "properties": {
    #         "short_description": {
    #           "type": "string",
    #           "description": "A brief description or label for the stored data. You don't need to write all the information in the description. The data you've searched for will be automatically stored in the Notebook."
    #         }
    #       },
    #       "required": ["short_description"]
    #     }
    #   }
    # },
    # {
    #   "type": "function",
    #   "function": {
    #     "name": "Planner",
    #     "description": "A smart planning tool that crafts detailed plans based on user input and the information stored in Notebook. For example, Planner('Give me a 3-day trip plan from Seattle to New York') would return a detailed 3-day trip plan.",
    #     "parameters": {
    #       "type": "object",
    #       "properties": {
    #         "query": {
    #           "type": "string",
    #           "description": "The query from user."
    #         }
    #       },
    #       "required": ["query"]
    #     }
    #   }
    # }
  ],
  tool_resources={
    "code_interpreter": {
      "file_ids": oai_file_ids
    }
  }
)

In [42]:
with open("./assistants.jsonl", 'a') as f:
    f.writelines([assistant.model_dump_json()])

In [43]:
assistant.id, assistant.description

('asst_lxFE0Q9nFmuN7tF54vHFXe65', 'You are a helpful travel planner')

### Create a thread and run

In [44]:
ASSISTANT_ID = "asst_lxFE0Q9nFmuN7tF54vHFXe65"
assistant = client.beta.assistants.retrieve(ASSISTANT_ID)

In [45]:
from typing_extensions import override
from openai import AssistantEventHandler

In [46]:
FUNC_NAME2API = {
    "FlightSearch": flight_api,
    "AccommodationSearch": accommodation_api,
    "AttractionSearch":  attraction_api,
    "RestaurantSearch": restaurant_api,
    "CitySearch": city_api,
    "DistanceMatrix": google_dist_api
}

In [47]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="Please help me plan a trip from St. Petersburg to Rockford spanning 3 days from March 16th to March 18th, 2022. The travel should be planned for a single person with a budget of $1,700.",
)

In [48]:
class EventHandler(AssistantEventHandler):
    @override
    def on_event(self, event):
      if event.event == 'thread.run.requires_action':
        run_id = event.data.id  # Retrieve the run ID from the event data
        self.handle_requires_action(event.data, run_id)
 
    def handle_requires_action(self, data, run_id):
      tool_outputs = []
        
      for tool in data.required_action.submit_tool_outputs.tool_calls:
        args = eval(tool.function.arguments)
        print(f"\n [env log]: call {tool.function.name} with argument {args} \n")
        api = FUNC_NAME2API[tool.function.name]
        result = api.run_for_mobi(**args)
        if result is None:
          tool_outputs.append({"tool_call_id": tool.id, "output": "no valid information"})
        elif type(result) is DataFrame:
          tool_outputs.append({"tool_call_id": tool.id, "output": result.to_json()})
        else:
          assert False, "return of api should be either DataFrame of None"
        
      # Submit all tool_outputs at the same time
      self.submit_tool_outputs(tool_outputs, run_id)
 
    def submit_tool_outputs(self, tool_outputs, run_id):
      # Use the submit_tool_outputs_stream helper
      with client.beta.threads.runs.submit_tool_outputs_stream(
        thread_id=self.current_run.thread_id,
        run_id=self.current_run.id,
        tool_outputs=tool_outputs,
        event_handler=EventHandler(),
      ) as stream:
        for text in stream.text_deltas:
          print(text, end="", flush=True)
        print()

In [49]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler()
) as stream:
  stream.until_done()

call FlightSearch with argument {'origin': 'St. Petersburg', 'destination': 'Rockford', 'departure_date': '2022-03-16'}
call AccommodationSearch with argument {'city': 'Rockford'}
### Flight Details
---
**Flight from St. Petersburg to Rockford:**
- **Flight Number:** F3573659
- **Price:** $474
- **Departure Time:** 15:40
- **Arrival Time:** 17:04
- **Duration:** 2 hours 24 minutes
- **Distance:** 1049 miles

### Accommodation Options in Rockford
---
Here is a list of available accommodations in Rockford:

1. **Spacious 3BDR Prime Location!**
   - **Type:** Entire home/apt
   - **Price:** $1,030 (No smoking)
   - **Minimum Nights:** 2
   - **Max Occupancy:** 9
   - **Review Rating:** 2

2. **Private Room in a two bedroom apt.**
   - **Type:** Private room
   - **Price:** $210 (No visitors & No smoking)
   - **Minimum Nights:** 1
   - **Max Occupancy:** 2
   - **Review Rating:** 4

3. **Private rooms And Matchless Location**
   - **Type:** Private room
   - **Price:** $1,075 (No pets & N

In [50]:
restaurant_api.run_for_mobi(**{'city': 'Rockford'})

,Name,Average Cost,Cuisines,Aggregate Rating,City
39,Coco Bambu,72,"Tea, French, Bakery, BBQ, Cafe",4.9,Rockford
251,Flying Mango,20,"American, BBQ, Seafood",4.5,Rockford
2470,Gajalee Sea Food,49,"Bakery, BBQ",3.9,Rockford
2861,Shree Balaji Chaat Bhandar,97,"French, Bakery, BBQ, Italian",3.2,Rockford
3052,Moets Arabica,43,"Tea, Bakery, Indian, Fast Food",3.5,Rockford
3163,Cafe Coffee Day,28,"Chinese, Desserts, Pizza, Cafe, Mediterranean",0.0,Rockford
3700,Nutri Punch,34,"Tea, Chinese, Cafe, Desserts",4.0,Rockford
4319,Cafe Southall,56,"Seafood, Pizza, Cafe, Fast Food",4.2,Rockford
4348,Eggspectation - Jaypee Vasant Continental,77,"Tea, Mediterranean, Seafood",3.6,Rockford
4542,Aroma Rest O Bar,58,"Bakery, Fast Food, Chinese, American, Seafood",3.6,Rockford
